In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.2 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [2]:
import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json


In [ ]:
import pandas as pd
import numpy as np
import os

# ==============================================================================
# CẤU HÌNH
# ==============================================================================
INPUT_FILE = '/content/drive/MyDrive/CAFA6_Results/ESM2_Run/submission.tsv'
OUTPUT_FILE = '/content/drive/MyDrive/CAFA6_Results/ESM2_Run/submission_CLEANED.tsv'

print(f"🕵️‍♂️ Đang kiểm tra file: {INPUT_FILE} ...")

# 1. KIỂM TRA ĐỌC FILE
try:
    # Đọc không header để kiểm soát toàn bộ dữ liệu
    df = pd.read_csv(INPUT_FILE, sep='\t', header=None, names=['id', 'term', 'score'])
    print(f"✅ Đọc file thành công. Tổng số dòng: {len(df):,}")
except Exception as e:
    print(f"❌ LỖI NGHIÊM TRỌNG: Không đọc được file. {e}")
    exit()

# Biến cờ để xem có cần lưu file mới không
need_fix = False

# 2. KIỂM TRA HEADER (Tiêu đề thừa)
# Nếu dòng đầu tiên chứa chữ "id" hoặc "term" hoặc "EntryID" -> Là header
first_row = df.iloc[0]
if str(first_row['score']).replace('.','',1).isdigit() is False:
    print(f"⚠️ CẢNH BÁO: Phát hiện dòng tiêu đề (Header): {first_row.values}")
    print("   -> Đang xóa dòng tiêu đề...")
    df = df.iloc[1:] # Bỏ dòng đầu
    # Ép kiểu lại cột score sang float
    df['score'] = pd.to_numeric(df['score'])
    need_fix = True
else:
    print("✅ Không có Header (Tốt).")

# 3. KIỂM TRA NAN (Giá trị rỗng)
nan_count = df.isna().sum().sum()
if nan_count > 0:
    print(f"⚠️ CẢNH BÁO: Phát hiện {nan_count} giá trị NaN (Rỗng).")
    print("   -> Đang xóa các dòng bị lỗi...")
    df = df.dropna()
    need_fix = True
else:
    print("✅ Không có NaN (Tốt).")

# 4. KIỂM TRA TRÙNG LẶP (DUPLICATES) - QUAN TRỌNG NHẤT
# Check xem có cặp (id, term) nào bị lặp lại không
dup_mask = df.duplicated(subset=['id', 'term'], keep=False)
dup_count = dup_mask.sum()

if dup_count > 0:
    print(f"🔴 CẢNH BÁO ĐỎ: Phát hiện {dup_count:,} dòng bị trùng lặp ID-Term!")
    print("   -> Đây chính là nguyên nhân gây Timeout/Crash.")
    print("   -> Đang xử lý: Giữ lại dòng có điểm số cao nhất, xóa dòng thấp hơn...")

    # Sort để điểm cao lên đầu -> Giữ thằng đầu tiên (keep='first')
    df = df.sort_values('score', ascending=False)
    df = df.drop_duplicates(subset=['id', 'term'], keep='first')

    print(f"   -> Đã xóa xong. Số dòng còn lại: {len(df):,}")
    need_fix = True
else:
    print("✅ Không có trùng lặp (Tuyệt vời).")

# 5. KIỂM TRA MIỀN GIÁ TRỊ (Score Range)
if pd.api.types.is_numeric_dtype(df['score']):
    min_score = df['score'].min()
    max_score = df['score'].max()
    print(f"ℹ️ Score Range: Min={min_score:.4f}, Max={max_score:.4f}")

    if min_score < 0 or max_score > 1.0:
        print("⚠️ CẢNH BÁO: Điểm số nằm ngoài khoảng [0, 1].")
        # Clip lại cho an toàn
        df['score'] = df['score'].clip(0.0, 1.0)
        need_fix = True
else:
    print("❌ LỖI: Cột Score không phải là số!")

# ==============================================================================
# KẾT LUẬN & LƯU FILE
# ==============================================================================
print("\n" + "="*50)
if need_fix:
    print("🛠️ ĐÃ SỬA CÁC LỖI TRÊN VÀ LƯU FILE MỚI.")
    print(f"📂 File sạch để nộp: {OUTPUT_FILE}")

    # Lưu file: tab-separated, không index, không header
    df.to_csv(OUTPUT_FILE, sep='\t', index=False, header=False, float_format='%.3f')

    # Check dung lượng
    size_mb = os.path.getsize(OUTPUT_FILE) / (1024*1024)
    print(f"📦 Dung lượng: {size_mb:.2f} MB")
    print("👉 HÃY NỘP FILE NÀY!")
else:
    print("✅ FILE CỦA BẠN HOÀN HẢO! KHÔNG CẦN SỬA GÌ CẢ.")
    print("👉 Nếu vẫn bị Timeout, hãy thử đổi tên file và nộp lại.")
print("="*50)

🕵️‍♂️ Đang kiểm tra file: /content/drive/MyDrive/CAFA6_Results/ESM2_Run/submission.tsv ...
✅ Đọc file thành công. Tổng số dòng: 4,404,027
✅ Không có Header (Tốt).
✅ Không có NaN (Tốt).
✅ Không có trùng lặp (Tuyệt vời).
ℹ️ Score Range: Min=0.0100, Max=0.9970

✅ FILE CỦA BẠN HOÀN HẢO! KHÔNG CẦN SỬA GÌ CẢ.
👉 Nếu vẫn bị Timeout, hãy thử đổi tên file và nộp lại.


In [ ]:
import os
import datetime

# ================= CẤU HÌNH =================
# Đường dẫn file gốc của bạn
OLD_FILE_PATH = '/content/drive/MyDrive/CAFA6_Results/ESM2_Run/submission.tsv'

# Nếu bạn đã chạy script sửa lỗi ở bước trước, hãy đổi dòng trên thành:
# OLD_FILE_PATH = '/content/drive/MyDrive/CAFA6_Results/ESM2_Run/submission_CLEANED.tsv'
# (Khuyên dùng file CLEANED để không bị Timeout)

# ================= XỬ LÝ =================
# 1. Tạo tên file mới có gắn thời gian (VD: submission_20240521_1030.tsv)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
folder = os.path.dirname(OLD_FILE_PATH)
new_filename = f"submission_{timestamp}.tsv"
NEW_FILE_PATH = os.path.join(folder, new_filename)

print(f"🔄 Đang copy file sang tên mới: {new_filename} ...")

# 2. Copy file (Dùng lệnh cp của Linux)
# Chúng ta dùng copy thay vì rename để giữ lại bản gốc dự phòng
os.system(f'cp "{OLD_FILE_PATH}" "{NEW_FILE_PATH}"')

if os.path.exists(NEW_FILE_PATH):
    print(f"✅ Đã tạo file mới tại: {NEW_FILE_PATH}")

    # 3. Gửi lệnh Submit lên Kaggle
    print("🚀 Đang gửi lệnh nộp bài...")
    message = f"esm2 top-k=20 top-k-labels=10k retry {timestamp}"

    # Chạy lệnh Kaggle CLI
    !kaggle competitions submit \
        -c cafa-6-protein-function-prediction \
        -f "$NEW_FILE_PATH" \
        -m "$message"

    print("\n✅ Đã gửi lệnh xong! Hãy check tab 'My Submissions' trên Kaggle.")
else:
    print("❌ Lỗi: Không tạo được file mới. Kiểm tra lại đường dẫn file gốc.")

🔄 Đang copy file sang tên mới: submission_20251202_143314.tsv ...
✅ Đã tạo file mới tại: /content/drive/MyDrive/CAFA6_Results/ESM2_Run/submission_20251202_143314.tsv
🚀 Đang gửi lệnh nộp bài...
100% 101M/101M [00:06<00:00, 16.2MB/s] 
400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/cafa-6-protein-function-prediction

✅ Đã gửi lệnh xong! Hãy check tab 'My Submissions' trên Kaggle.


In [3]:
# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
print("⏳ Đang tải dữ liệu CAFA 6...")
!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

⏳ Đang tải dữ liệu CAFA 6...
  0% 0.00/91.2M [00:00<?, ?B/s]
100% 91.2M/91.2M [00:00<00:00, 1.28GB/s]
✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data


In [ ]:
import pandas as pd
import numpy as np
import os
import requests
from Bio import SeqIO

# ==============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ==============================================================================
# 1. File Submission của bạn
SUBMISSION_FILE = '/content/drive/MyDrive/CAFA6_Results/ESM2_Run/submission.tsv'

# 2. File chứa ID thật của tập Test (FASTA)
TEST_FASTA = '/content/cafa6_data/Test/testsuperset.fasta'

# 3. File định nghĩa Gene Ontology (Sẽ tự tải nếu chưa có)
GO_OBO_URL = "http://purl.obolibrary.org/obo/go/go-basic.obo"
GO_OBO_PATH = "/content/cafa6_data/go-basic.obo"

# ==============================================================================
# HÀM HỖ TRỢ
# ==============================================================================

def download_obo_if_needed(url, path):
    if not os.path.exists(path):
        print(f"📥 Đang tải file go-basic.obo từ {url} ...")
        r = requests.get(url, allow_redirects=True)
        with open(path, 'wb') as f:
            f.write(r.content)
        print("✅ Đã tải xong.")
    else:
        print("✅ Đã tìm thấy file go-basic.obo.")

def load_valid_go_terms(obo_path):
    print("⏳ Đang đọc file OBO để lấy danh sách GO hợp lệ...")
    valid_terms = set()
    with open(obo_path, 'r') as f:
        current_term = None
        for line in f:
            line = line.strip()
            if line == "[Term]":
                current_term = ""
            elif line.startswith("id: GO:"):
                if current_term is not None:
                    valid_terms.add(line.split('id: ')[1])
            elif line.startswith("alt_id: GO:"):
                # Chấp nhận cả ID thay thế
                if current_term is not None:
                    valid_terms.add(line.split('alt_id: ')[1])
    print(f"   -> Đã load {len(valid_terms):,} nhãn GO hợp lệ.")
    return valid_terms

# ==============================================================================
# CHƯƠNG TRÌNH KIỂM TRA
# ==============================================================================
print("🚀 BẮT ĐẦU KIỂM TRA TÍNH HỢP LỆ CỦA SUBMISSION")
print("="*60)

# 1. Đọc file Submission
print(f"📂 Đang đọc submission: {SUBMISSION_FILE}")
try:
    df = pd.read_csv(SUBMISSION_FILE, sep='\t', header=None, names=['id', 'term', 'score'])
    # Nếu file có header vô tình, xóa nó đi
    if not str(df.iloc[0]['score']).replace('.','',1).isdigit():
        df = df.iloc[1:]

    sub_ids = set(df['id'].unique())
    sub_terms = set(df['term'].unique())
    print(f"   -> Tìm thấy {len(sub_ids):,} proteins và {len(sub_terms):,} GO terms trong file nộp.")
except Exception as e:
    print(f"❌ LỖI: Không đọc được file submission. {e}")
    exit()

# 2. Kiểm tra Protein IDs (So với file Fasta)
print("\n🔍 CHECK 1: KIỂM TRA PROTEIN IDs")
print("-" * 30)
try:
    valid_test_ids = set()
    for record in SeqIO.parse(TEST_FASTA, "fasta"):
        valid_test_ids.add(record.id.split()[0]) # Lấy ID chính

    print(f"   -> Tập Test chuẩn có {len(valid_test_ids):,} proteins.")

    # Tìm ID lạ (Có trong file nộp nhưng KHÔNG CÓ trong đề thi)
    unknown_ids = sub_ids - valid_test_ids
    if len(unknown_ids) > 0:
        print(f"⚠️ CẢNH BÁO: Có {len(unknown_ids):,} ID trong file nộp KHÔNG TỒN TẠI trong tập Test!")
        print(f"   -> Ví dụ: {list(unknown_ids)[:5]}")
        print("   -> LÝ DO: Có thể do chưa clean ID (vẫn còn 'sp|...|') hoặc nhầm lẫn tập train/test.")
    else:
        print("✅ TẤT CẢ Protein ID đều hợp lệ.")

    # Tìm ID bỏ sót (Có trong đề thi nhưng KHÔNG CÓ trong file nộp)
    missing_ids = valid_test_ids - sub_ids
    if len(missing_ids) > 0:
        print(f"ℹ️ Lưu ý: Bạn chưa dự đoán cho {len(missing_ids):,} proteins trong tập test.")
        print("   (Điều này không lỗi, nhưng sẽ làm giảm Recall).")
    else:
        print("✅ Bạn đã dự đoán đầy đủ cho 100% tập test.")

except Exception as e:
    print(f"❌ Lỗi khi đọc file Fasta: {e}")

# 3. Kiểm tra GO Terms (So với go-basic.obo)
print("\n🔍 CHECK 2: KIỂM TRA GO TERMS")
print("-" * 30)
try:
    download_obo_if_needed(GO_OBO_URL, GO_OBO_PATH)
    valid_go_set = load_valid_go_terms(GO_OBO_PATH)

    # Tìm nhãn "Ma" (Ghost Terms) - Có nộp nhưng không tồn tại trong từ điển
    invalid_terms = sub_terms - valid_go_set

    if len(invalid_terms) > 0:
        print(f"🔴 LỖI NGHIÊM TRỌNG: Có {len(invalid_terms):,} nhãn GO không hợp lệ (không có trong OBO)!")
        print(f"   -> Ví dụ: {list(invalid_terms)[:5]}")
        print("   -> HẬU QUẢ: Các dòng này sẽ bị coi là rác và không được tính điểm.")
    else:
        print("✅ TẤT CẢ nhãn GO đều hợp lệ (Tồn tại trong go-basic.obo).")

except Exception as e:
    print(f"❌ Lỗi khi kiểm tra OBO: {e}")

print("\n" + "="*60)
print("🏁 KẾT THÚC KIỂM TRA")

🚀 BẮT ĐẦU KIỂM TRA TÍNH HỢP LỆ CỦA SUBMISSION
📂 Đang đọc submission: /content/drive/MyDrive/CAFA6_Results/ESM2_Run/submission.tsv
   -> Tìm thấy 224,309 proteins và 9,231 GO terms trong file nộp.

🔍 CHECK 1: KIỂM TRA PROTEIN IDs
------------------------------
   -> Tập Test chuẩn có 224,309 proteins.
✅ TẤT CẢ Protein ID đều hợp lệ.
✅ Bạn đã dự đoán đầy đủ cho 100% tập test.

🔍 CHECK 2: KIỂM TRA GO TERMS
------------------------------
📥 Đang tải file go-basic.obo từ http://purl.obolibrary.org/obo/go/go-basic.obo ...
✅ Đã tải xong.
⏳ Đang đọc file OBO để lấy danh sách GO hợp lệ...
   -> Đã load 51,842 nhãn GO hợp lệ.
✅ TẤT CẢ nhãn GO đều hợp lệ (Tồn tại trong go-basic.obo).

🏁 KẾT THÚC KIỂM TRA


In [4]:
import pandas as pd

# Đường dẫn file IA (Sửa lại nếu đường dẫn của bạn khác)
IA_PATH = '/content/cafa6_data/IA.tsv'

print(f"📂 Đang đọc file: {IA_PATH} ...")

try:
    # File IA thường không có header, cột 1 là Term, cột 2 là Weight
    df = pd.read_csv(IA_PATH, sep='\t', names=['term', 'ia'])

    # Thống kê
    total_terms = len(df)
    positive_ia = df[df['ia'] > 0]
    count_positive = len(positive_ia)

    print("-" * 50)
    print(f"✅ Tổng số Terms trong từ điển: {total_terms:,}")
    print(f"🎯 Số Terms có weight > 0:     {count_positive:,}")
    print(f"📊 Tỷ lệ terms có thông tin:    {(count_positive / total_terms) * 100:.2f}%")
    print("-" * 50)

    # In thử top 5 term có trọng số cao nhất (khó nhất)
    print("\nTop 5 terms có IA cao nhất (Khó dự đoán nhất):")
    print(positive_ia.sort_values(by='ia', ascending=False).head(5))

except FileNotFoundError:
    print("❌ Lỗi: Không tìm thấy file. Hãy kiểm tra lại đường dẫn.")

📂 Đang đọc file: /content/cafa6_data/IA.tsv ...
--------------------------------------------------
✅ Tổng số Terms trong từ điển: 40,122
🎯 Số Terms có weight > 0:     22,078
📊 Tỷ lệ terms có thông tin:    55.03%
--------------------------------------------------

Top 5 terms có IA cao nhất (Khó dự đoán nhất):
             term         ia
29382  GO:0120280  14.861014
27127  GO:0031912  14.861014
38857  GO:0102910  14.823815
39821  GO:0180024  14.823815
34769  GO:0035275  14.413991
